In [ ]:
# 基础包
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from Random_Erasing.random_erasing import RandomErasing
import torchvision.datasets as datasets
import numpy as np
# 架构
# 导入所有模型架构
from wideresnet import WideResNet
from wideresnet_CDD import WideResNet_CDD
from resnet import Resnet
from resnet_CDD import Resnet_CDD
from senet import Senet
from senet_CDD import Senet_CDD
from se_resnet import Se_net
from se_resnet_CDD import Se_net_CDD
from densenet3 import DenseNet
from densenet3_CDD import DenseNet_CDD
from shake_shake import Shake_shake
from shake_shake_CDD import Shake_shake_CDD
# 运行设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# In[1] 定义测试函数
def test(model,test_loader):
    
    model.eval() #切换到评估模式
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs= model(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc=100 * correct / total
    
        print('Accuracy of the model on the test images: {} %'.format(acc))
    return acc

def best_prec(best_prec, path, name):
    best_prec = np.array(best_prec)
    try:
        precmax = np.load(path+'/'+name+'.npy')
        
    except:
        precmax = 0
        precmax = np.array(precmax)
    
    if precmax < best_prec:
        precmax = best_prec
        np.save(path+'/'+name+'.npy', precmax)
    print('the best Prediction accuracy of',name,'is:', precmax)

def Robust_test(model, dataset, path):
    batch_size = 512
    # 归一化，白化参数
    normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])
    # NO_normalize
    transform_test = transforms.Compose([
        transforms.ToTensor(),                # 张量
        ])
    kwargs = {'num_workers': 1, 'pin_memory': True}
    val_loader = torch.utils.data.DataLoader(
        datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
        batch_size=batch_size, shuffle=True, **kwargs)
    prec_NO_n = test(model,val_loader)
    best_prec(prec_NO_n, path, 'prec_NO_n')
    print('NO_normalize：', prec_NO_n)
    # 干扰办法
    transform_test = transforms.Compose([
        transforms.ToTensor(),                # 张量
        normalize,                            # 归一化，白化
        RandomErasing(),                      # 随机遮挡
        ])
    kwargs = {'num_workers': 1, 'pin_memory': True}
    val_loader = torch.utils.data.DataLoader(
        datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
        batch_size=batch_size, shuffle=True, **kwargs)
    prec_RE = test(model,val_loader)
    best_prec(prec_RE, path, 'prec_RE')
    print('随机遮挡的精度：', prec_RE)

    # 干扰办法
    transform_test = transforms.Compose([
        transforms.ColorJitter(brightness=0.5, hue=0.5, contrast=0.5, saturation=0.5), # 随机改变图片的亮度brightness，色调hue，对比度contrast，饱和度saturation
        transforms.ToTensor(),                # 张量
        normalize,                            # 归一化，白化
        ])
    val_loader = torch.utils.data.DataLoader(
        datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
        batch_size=batch_size, shuffle=True, **kwargs)
    prec_CJ = test(model,val_loader)
    best_prec(prec_CJ, path, 'prec_CJ')
    print('随机调节亮度brightness，色调hue，对比度contrast，饱和度saturation的精度：', prec_CJ)
    # 干扰办法
    transform_test = transforms.Compose([
        transforms.RandomHorizontalFlip(),    # 随机水平翻转
        transforms.RandomVerticalFlip(),      # 随机竖直翻转
        transforms.ToTensor(),                # 张量
        normalize,                            # 归一化，白化
        ])

    val_loader = torch.utils.data.DataLoader(
        datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
        batch_size=batch_size, shuffle=True, **kwargs)
    prec_RVF = test(model,val_loader)
    best_prec(prec_RVF, path, 'prec_RVF')
    print('随机水平和竖直翻转的精度：', prec_RVF)

def main():
    # 定义一个模型的架构
    model = WideResNet_CDD(depth=40, num_classes=10, widen_factor=4, dropRate=0.0).to(device)  # 对应的是WRN40_4和cifar10数据集
    # 参数
    # 导入你想要测试的模型参数
    path = "./cifar10_WideResNet_CDD_40_4/model.pth"   # 模型参数的位置
    model.load_state_dict(torch.load(path))
    dataset = 'CIFAR10'
    # dataset = 'CIFAR100'
    Robust_test(model, dataset, path)

if __name__ == '__main__':
    main()

cuda
Accuracy of the model on the test images: 97.09 %
clean accuracy： 97.09
Accuracy of the model on the test images: 81.5 %
NO_normalize： 81.5
Accuracy of the model on the test images: 94.87 %
随机遮挡的精度： 94.87
Accuracy of the model on the test images: 87.76 %
随机调节亮度brightness，色调hue，对比度contrast，饱和度saturation的精度： 87.76


In [5]:
# 基础包
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from Random_Erasing.random_erasing import RandomErasing
import torchvision.datasets as datasets
# 架构
# 导入所有模型架构
from wideresnet import WideResNet
from wideresnet_CDD import WideResNet_CDD
from resnet import Resnet
from resnet_CDD import Resnet_CDD
from senet import Senet
from senet_CDD import Senet_CDD
from se_resnet import Se_net
from se_resnet_CDD import Se_net_CDD
from densenet3 import DenseNet
from densenet3_CDD import DenseNet_CDD
from shake_shake import Shake_shake
from shake_shake_CDD import Shake_shake_CDD
# 运行设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 定义一个模型的架构
model = WideResNet(depth=40, num_classes=10, widen_factor=4, dropRate=0.0).to(device)  # 对应的是WRN40_4和cifar10数据集
# 参数
# 导入你想要测试的模型参数
path = "./cifar10_WideResNet_40_4/model.pth"   # 模型参数的位置
model.load_state_dict(torch.load(path))
dataset = 'CIFAR10'
# dataset = 'CIFAR100'
batch_size = 512
# 归一化，白化参数
normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                 std=[x/255.0 for x in [63.0, 62.1, 66.7]])
# In[1] 定义测试函数
def test(model,test_loader):
    
    model.eval() #切换到评估模式
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs= model(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc=100 * correct / total
    
        print('Accuracy of the model on the test images: {} %'.format(acc))
    return acc
# clean accuracy
transform_test = transforms.Compose([
    transforms.ToTensor(),                # 张量
    normalize,                            # 归一化，白化
    ])
kwargs = {'num_workers': 1, 'pin_memory': True}
val_loader = torch.utils.data.DataLoader(
    datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, **kwargs)
prec_CL = test(model,val_loader)
print('clean accuracy：', prec_CL)
# NO_normalize
transform_test = transforms.Compose([
    transforms.ToTensor(),                # 张量
    ])
kwargs = {'num_workers': 1, 'pin_memory': True}
val_loader = torch.utils.data.DataLoader(
    datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, **kwargs)
prec_NO_n = test(model,val_loader)
print('NO_normalize：', prec_NO_n)
# 干扰办法
transform_test = transforms.Compose([
    transforms.ToTensor(),                # 张量
    normalize,                            # 归一化，白化
    RandomErasing(),                      # 随机遮挡
    ])
kwargs = {'num_workers': 1, 'pin_memory': True}
val_loader = torch.utils.data.DataLoader(
    datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, **kwargs)
prec_RE = test(model,val_loader)
print('随机遮挡的精度：', prec_RE)

# 干扰办法
transform_test = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, hue=0.5, contrast=0.5, saturation=0.5), # 随机改变图片的亮度brightness，色调hue，对比度contrast，饱和度saturation
    transforms.ToTensor(),                # 张量
    normalize,                            # 归一化，白化
    ])
val_loader = torch.utils.data.DataLoader(
    datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, **kwargs)
prec_CJ = test(model,val_loader)
print('随机调节亮度brightness，色调hue，对比度contrast，饱和度saturation的精度：', prec_CJ)
# 干扰办法
transform_test = transforms.Compose([
    transforms.RandomHorizontalFlip(),    # 随机水平翻转
    # transforms.RandomVerticalFlip(),      # 随机竖直翻转
    transforms.ToTensor(),                # 张量
    normalize,                            # 归一化，白化
    ])

val_loader = torch.utils.data.DataLoader(
    datasets.__dict__[dataset.upper()]('./data', train=False, transform=transform_test),
    batch_size=batch_size, shuffle=True, **kwargs)
prec_RVF = test(model,val_loader)
print('随机水平和竖直翻转的精度：', prec_RVF)


cuda
Accuracy of the model on the test images: 96.99 %
clean accuracy： 96.99
Accuracy of the model on the test images: 78.66 %
NO_normalize： 78.66
Accuracy of the model on the test images: 94.84 %
随机遮挡的精度： 94.84
Accuracy of the model on the test images: 87.26 %
随机调节亮度brightness，色调hue，对比度contrast，饱和度saturation的精度： 87.26
Accuracy of the model on the test images: 96.98 %
随机水平和竖直翻转的精度： 96.98
